# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298537830893                   -0.85    5.0         
  2   -8.300215905718       -2.78       -1.26    1.0    123ms
  3   -8.300437963771       -3.65       -1.89    3.0    161ms
  4   -8.300461509763       -4.63       -2.78    2.1    137ms
  5   -8.300464479126       -5.53       -3.15    2.9    176ms
  6   -8.300464572855       -7.03       -3.34    1.8    159ms
  7   -8.300464606789       -7.47       -3.49    1.6    179ms
  8   -8.300464626446       -7.71       -3.64    2.1    151ms
  9   -8.300464637371       -7.96       -3.80   12.2    308ms
 10   -8.300464641542       -8.38       -3.95    1.8    151ms
 11   -8.300464643793       -8.65       -4.30    2.9    174ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.63205098771                   -0.70    5.4         
  2   -16.63609604002       -2.39       -1.14    1.0    328ms
  3   -16.67915216157       -1.37       -1.72    3.0    396ms
  4   -16.67927252979       -3.92       -2.23    2.6    399ms
  5   -16.67928465965       -4.92       -2.85    3.2    424ms
  6   -16.67928604750       -5.86       -3.15    4.6    480ms
  7   -16.67928615566       -6.97       -3.29    1.1    295ms
  8   -16.67928621475       -7.23       -3.85    1.0    285ms
  9   -16.67928622144       -8.17       -4.47    2.8    390ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.31706541185                   -0.56    6.5         
  2   -33.32570574027       -2.06       -1.00    1.4    1.27s
  3   -33.33594185867       -1.99       -1.68    6.9    1.78s
  4   -33.33684222647       -3.05       -2.33    4.5    1.66s
  5   -33.33693478949       -4.03       -2.83    6.8    1.89s
  6   -33.33694352733       -5.06       -3.58    3.9    1.56s
  7   -33.33694375895       -6.64       -3.93    4.2    1.62s
  8   -33.33694377955       -7.69       -4.59    2.4    1.32s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298500142106                   -0.85    5.0         
  2   -8.300276831839       -2.75       -1.59    1.0   99.2ms
  3   -8.300429112481       -3.82       -2.57    3.2    142ms
  4   -8.300388609159   +   -4.39       -2.31    4.1    209ms
  5   -8.300464159114       -4.12       -3.37    1.1    109ms
  6   -8.300464550196       -6.41       -3.72    3.9    183ms
  7   -8.300464637949       -7.06       -4.21    1.0    112ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32397914175                   -0.56    6.5         
  2   -33.29938938023   +   -1.61       -1.26    1.0    988ms
  3   -0.562954189508   +    1.52       -0.31    6.8    2.34s
  4   -33.33227545044        1.52       -1.87    6.4    2.13s
  5   -32.91479577789   +   -0.38       -1.18    5.0    2.02s
  6   -32.27365940249   +   -0.19       -1.04    4.6    1.85s
  7   -33.32307297669        0.02       -1.88    4.2    1.70s
  8   -33.32826506182       -2.28       -1.91    2.0    1.23s
  9   -33.33516197916       -2.16       -2.14    2.1    1.28s
 10   -33.33673779456       -2.80       -2.76    1.6    1.13s
 11   -33.33691813688       -3.74       -3.15    3.5    1.61s
 12   -33.33692776329       -5.02      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.